In [1]:
import tools.architectures as nn
import tools.processing as pre
import tensorflow as tf

run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

text = pre.get_text("data/prepped/clean2_pac.txt")[:30000]
vocab = pre.Vocabulary(text)

class MultiLayerEmbeddedLabels(nn.Trainable):
    def __init__(self, name):
        super().__init__(name)

    def build(self, num_layers, hidden_layer_size, vocab_size, time_steps, embedding):
        self.time_steps = time_steps
        self.vocab_size = vocab_size

        # self.Y = tf.placeholder(tf.int32, shape=[None, 1], name="labels")
        self.Y = tf.placeholder(tf.int32, shape=[None, vocab_size], name="labels")

        self.X = tf.placeholder(tf.int32, shape=[None, time_steps], name="data")
        self.embed, self.embeddings = embedding.output(self.X)


        with tf.variable_scope(self.name, reuse=tf.AUTO_REUSE):

            self.stacked_cells = nn.lstm_layer(num_layers, hidden_layer_size)

            self.outputs, self.states = tf.nn.dynamic_rnn(
                self.stacked_cells, self.embed, dtype=tf.float32
            )

            self.last_rnn_output = self.states[num_layers - 1][1]

            self.embedded_output, W_out, b_out = nn.full_layer(self.last_rnn_output, embedding.embedding_dimension)

            self.weights.append(W_out)
            self.biases.append(b_out)
            
            # self.embed_labels  = tf.nn.embedding_lookup( self.embeddings, tf.argmax(self.Y, 1) )
            # self.embed_preds   = tf.nn.embedding_lookup( self.embeddings, self.final_output )
            emb_distances = tf.matmul( # shape == (vocab_size, batch_size)
                tf.nn.l2_normalize(self.embeddings, axis=1),
                tf.nn.l2_normalize(self.embedded_output, axis=1),
                transpose_b=True)
            # token_ids = tf.argmax(emb_distances, axis=0) # shape == (batch_size)
            self.final_output  = tf.transpose(emb_distances)
            
            # self.loss = tf.reduce_mean(tf.square(self.embedded_output - self.embed_labels))
            self.softmax = tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.final_output,
                    labels=self.Y)
            self.cross_entropy_loss = tf.reduce_mean(self.softmax)
            self.loss = self.cross_entropy_loss

            self.optimizer = tf.train.AdamOptimizer()
            self.train_step= self.optimizer.minimize(self.loss)

            self.correct_prediction = tf.equal(tf.argmax(self.Y,1), tf.argmax(self.final_output, 1))
            self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))*100

In [2]:
text = pre.get_text("data/prepped/clean2_pac.txt")[:60000]
vocab = pre.Vocabulary(text)

NUM_LAYERS = 2
HIDDEN_LAYER_SIZE = 512
VOCAB_SIZE = vocab.get_size()
TIMESTEPS = 20

EPOCHS = 20
BATCH_SIZE = 256

EMBEDDING_SIZE = 128


data, labels = vocab.making_embedded_one_hot(text, TIMESTEPS)
# data, labels = vocab.making_full_one_hot(text, TIMESTEPS)

embedding = nn.LeanableEmbedding(name = "learnable-embedding-2")
embedding.build(VOCAB_SIZE, EMBEDDING_SIZE)

rnn = MultiLayerEmbeddedLabels(name = "multi-pac-total")
rnn.build(NUM_LAYERS, HIDDEN_LAYER_SIZE, VOCAB_SIZE, TIMESTEPS, embedding)

nn.train(rnn, data, labels, vocab, epochs=EPOCHS, batch_size=BATCH_SIZE, temperature=.6, embedding=True)



Epoch 1/20
Loss:    	 7.489138126373291
Accuracy:	 9.765625
as real as it seems the american dream
is not nothing but another calculated schemes
to get us
Seed:as real as it seems the american dream 
 is not nothing but another calculated schemes 
 to get us 
Result:as real as it seems the american dream 
 is not nothing but another calculated schemes 
 to get us raw sitting cute crates cigarettes fact write b undressed picked slacks expand yourself pain tick learning mutule accomplished host impossible grace pretend comes peewee fiend martin history about habit mad intro cousins fight within oppressor audience talking peach fling intends somewhere error other settle sweaty rhymefumble terror swinging fatter prepared penny battle bodies geonim tying san hype seconds lightly bodyguard civilized act house win gunfight colorin is conciser bond tumble count flip park trick wear playing motts ask rush wanted nightstand pal fame bed rosin city my geonim named today conceal packs impossible

KeyboardInterrupt: 

In [4]:
labels.shape

(13921, 2099)

In [5]:
data.shape

(13921, 20)

In [3]:

_data, _labels = vocab.making_full_one_hot(text, 20)


In [4]:
_labels

array([[  18.],
       [  19.],
       [  20.],
       ...,
       [1236.],
       [   7.],
       [1253.]])

In [1]:
import tools.architectures as nn
import tools.processing as pre

text = pre.get_text("data/prepped/clean2_pac.txt")[:30000]
vocab = pre.Vocabulary(text)

#print(corrected.replace(" lbreak ", "\n"))
# processed
# text = corrected.replace(" lbreak ", "\n")

vocab = pre.Vocabulary(text)

NUM_LAYERS = 2
HIDDEN_LAYER_SIZE = 512
VOCAB_SIZE = vocab.get_size()
TIMESTEPS = 15

EPOCHS = 30
BATCH_SIZE = 256

EMBEDDING_SIZE = 10


data, labels = vocab.making_embedded_one_hot(text, TIMESTEPS)

embedding = nn.LeanableEmbedding(name = "learnable-embedding")
embedding.build(VOCAB_SIZE, EMBEDDING_SIZE)

rnn = nn.MultiLayerRNN(name = "single-pac")
rnn.build(NUM_LAYERS, HIDDEN_LAYER_SIZE, VOCAB_SIZE, TIMESTEPS, l2_reg=0.0, embedding=embedding)

nn.train(rnn, data, labels, vocab, epochs=EPOCHS, batch_size=BATCH_SIZE, temperature=.6, embedding=True)



Epoch 1/30
Loss:    	 5.988228797912598
Accuracy:	 10.9375
as real as it seems the american dream
is not nothing but another calculated
Seed:as real as it seems the american dream 
 is not nothing but another calculated 
Result:as real as it seems the american dream 
 is not nothing but another calculated 
 
 
 is 
 i 
 of 
 
 
 
 now groin but 
 
 i 
 
 i i it my 
 told it 
 
 
 
 
 
 
 
 it 
 
 
 i 
 seen 
 
 i am 
 
 
 
 am i want night am 
 
 it 
 
 
 to and are 
 a 
 up 
 
 
 could to it 
 if the 
 
 
 is 
 that is 
 
 you i 
 
 and i 
 
 it 
 
 my position a 
 i 
 the 
 
 
 
 i the 
 i 
 
 
 
 i 
 i are hic i and but 
 
 
 i i dope you 
 
 i 
 of that i 
 
 
 
 but i to i 
 when all 
 
 
 i 
 you 
 i get 
 this 
 to be 
 i 
 will but 
 
 busy i 
 cause 
 you 
 
 
 with 
 i 
 it my i 
 
 
 i i 
 
 
 
 to 
 muttered 
 
 i 
 
 enjoyed 
 
 i cause i 
 
 
 
 will my i 
 
 tried the 
 my 
 
 
 
 
 you i i 
 to well do 
 i i the i 
 i 
 and 
 dizzy 
 musician i i that 
 
 to suddenly 